In [2]:
import pandas as pd
import numpy as np

In [13]:
codes = pd.read_csv('Airline_Code.csv')
codes.head()

,web-scraper-order,web-scraper-start-url,IATA,ICAO,Airline,Call sign,Country_Region,Comments
0,1733246192-1,https://en.wikipedia.org/wiki/List_of_airline_...,PR,BOI,2GO,ABAIR,Philippines,NaN
1,1733246192-2,https://en.wikipedia.org/wiki/List_of_airline_...,NaN,EVY,"34 Squadron, Royal Australian Air Force",Multiple,NaN,NaN
2,1733246192-3,https://en.wikipedia.org/wiki/List_of_airline_...,NaN,GNL,135 Airways,GENERAL,United States,NaN
3,1733246192-4,https://en.wikipedia.org/wiki/List_of_airline_...,1A,NaN,Amadeus IT Group S.A.,NaN,Global,GDS and airline hosting system (CRS/PSS)
4,1733246192-5,https://en.wikipedia.org/wiki/List_of_airline_...,1B,NaN,Sabre travel network Asia-Pacific (ex-Abacus),NaN,APAC,Regional distribution


In [25]:
codes.__len__()

758

In [22]:
codes = codes.dropna(subset=['IATA'])
codes = codes[~codes['Comments'].str.contains('defunct', na=False)]
codes = codes[~codes['Comments'].str.contains('Defunct', na=False)]
codes = codes[~codes['Airline'].str.contains('Cargo', na=False)]
codes = codes[~codes['Airline'].str.contains('Limited', na=False)]
codes = codes[~codes['Airline'].str.contains('Contractors', na=False)]
codes = codes[~codes['Airline'].str.contains('Logistic', na=False)]
codes = codes[~codes['IATA'].str.contains(r'\d')]
codes = codes.sort_values(by='IATA', ascending=True)
codes.head(10)

,web-scraper-order,web-scraper-start-url,IATA,ICAO,Airline,Call sign,Country_Region,Comments
1501,1733246192-1502,https://en.wikipedia.org/wiki/List_of_airline_...,AA,AAL,American Airlines,AMERICAN,United States,NaN
412,1733246192-413,https://en.wikipedia.org/wiki/List_of_airline_...,AC,ACA,Air Canada,AIR CANADA,Canada,NaN
1538,1733246192-1539,https://en.wikipedia.org/wiki/List_of_airline_...,AD,AZU,Azul Linhas Aéreas Brasileiras,Azul,Brazil,NaN
3937,1733246192-3938,https://en.wikipedia.org/wiki/List_of_airline_...,AE,MDA,Mandarin Airlines,MANDARIN,Taiwan,NaN
285,1733246192-286,https://en.wikipedia.org/wiki/List_of_airline_...,AF,AFR,Air France,AIRFRANS,France,NaN
1334,1733246192-1335,https://en.wikipedia.org/wiki/List_of_airline_...,AG,ARU,Aruba Airlines,ARUBA,Aruba,NaN
1511,1733246192-1512,https://en.wikipedia.org/wiki/List_of_airline_...,AG,ABR,ASL Airlines Ireland,CONTRACT,Ireland,Former IATA Code: AG; former names: Hunting Ai...
969,1733246192-970,https://en.wikipedia.org/wiki/List_of_airline_...,AH,DAH,Air Algérie,AIR ALGERIE,Algeria,NaN
776,1733246192-777,https://en.wikipedia.org/wiki/List_of_airline_...,AK,AXM,AirAsia,RED CAP,Malaysia,ICAO code no longer allocated
5291,1733246192-5292,https://en.wikipedia.org/wiki/List_of_airline_...,AL,SYX,Skywalk Airlines,SKYWAY-EX,United States,(Astral Aviation)


In [31]:
duplicate_iata_count = codes['IATA'].duplicated().sum()
print(f"Number of duplicate IATA codes: {duplicate_iata_count}")

Number of duplicate IATA codes: 0


In [30]:
duplicate_iata = codes[codes.duplicated(subset=['IATA'], keep=False)]
print(duplicate_iata)

Empty DataFrame
Columns: [web-scraper-order, web-scraper-start-url, IATA, ICAO, Airline, Call sign, Country_Region, Comments]
Index: []


In [29]:
import ipywidgets as widgets
from IPython.display import display

# Function to create selection widgets for each conflicting IATA code
def create_selection_widgets(conflicting_iata_codes, duplicate_iata):
    selection_widgets = {}
    for iata in conflicting_iata_codes:
        conflicting_entries = duplicate_iata[duplicate_iata['IATA'] == iata]
        options = [(entry['Airline'], entry.name) for _, entry in conflicting_entries.iterrows()]
        dropdown = widgets.Dropdown(
            options=options,
            value=options[0][1],  # Select the first airline by default
            description=f'IATA {iata}:'
        )
        selection_widgets[iata] = dropdown
    return selection_widgets

# Function to display the selection widgets and commit button
def display_selection_widgets(selection_widgets):
    items = [widget for widget in selection_widgets.values()]
    commit_button = widgets.Button(description='Commit Changes', button_style='success')
    display(widgets.VBox(items + [commit_button]))
    return commit_button

# Function to handle the commit button click event
def on_commit_button_clicked(b):
    for iata, dropdown in selection_widgets.items():
        selected_entry = dropdown.value
        codes.drop(codes[(codes['IATA'] == iata) & (codes.index != selected_entry)].index, inplace=True)
    print("Conflicting entries have been resolved and unnecessary airlines have been removed.")

# Create and display the selection widgets
selection_widgets = create_selection_widgets(conflicting_iata_codes, duplicate_iata)
commit_button = display_selection_widgets(selection_widgets)

# Attach the click event handler to the commit button
commit_button.on_click(on_commit_button_clicked)

Conflicting entries have been resolved and unnecessary airlines have been removed.


In [ ]:
codes = codes.drop(columns=['web-scraper-order', 'web-scraper-start-url', 'ICAO', 'Call sign', 'Comments'])
codes = codes.dropna(subset=['Country_Region'])
codes['IATA'] = codes['IATA'].str.slice(0, 2)
codes.head()

,IATA,Airline,Country_Region
1501,AA,American Airlines,United States
412,AC,Air Canada,Canada
1538,AD,Azul Linhas Aéreas Brasileiras,Brazil
3937,AE,Mandarin Airlines,Taiwan
285,AF,Air France,France


In [40]:
# Export the cleaned data to a new CSV file
codes.to_csv('Airline_Code_Cleaned.csv', index=False)

In [4]:
# Import the cleaned CSV file
cleaned_codes = pd.read_csv('Airline_Code_Cleaned.csv')

# Format each row for SQL entry
formatted_rows = []
for index, row in cleaned_codes.iterrows():
    formatted_row = f"INSERT INTO airline VALUES('{row['IATA']}', '{row['Airline']}', '{row['Country_Region']}')"
    formatted_rows.append(formatted_row)

# Join all formatted rows into a single string
formatted_text = ';\n'.join(formatted_rows)

# Export the formatted text to a plaintext file
with open('Airline_Code_SQL_Entries.txt', 'w') as file:
    file.write(formatted_text)

ENG MFR MDL,STREET,STREET2,CITY,STATE,ZIP CODE,REGION,COUNTY,LAST ACTION DATE,CERT ISSUE DATE,CERTIFICATION,TYPE AIRCRAFT,TYPE ENGINE,STATUS CODE,MODE S CODE,FRACT OWNER,AIR WORTH DATE,OTHER NAMES(1),OTHER NAMES(2),OTHER NAMES(3),OTHER NAMES(4),OTHER NAMES(5),EXPIRATION DATE,UNIQUE ID,KIT MFR, KIT MODEL,MODE S CODE HEX,